## Generative Latent Replay with GMMs

Code to test latent replay on benchmark problems and compare with proposed generative latent replay strategies i.e. normalising bottleneck representations and sampling from fitted GMM on latent space.

In [ ]:
#!pip install git+https://github.com/iacobo/avalanche.git

In [1]:
import torch
from torch.nn import CrossEntropyLoss
from torch.optim import SGD

from avalanche.benchmarks.classic import PermutedMNIST #,PermutedOmniglot, RotatedOmniglot
from avalanche.models import SimpleMLP
from avalanche.training import Naive#, LatentReplay
from avalanche.training.supervised.latent_replay import LatentReplay

from torchvision import transforms

In [5]:
# Config
device = "cpu" #torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# model
model = SimpleMLP(num_classes=10)

# CL Benchmark Creation
# Original AR1* usese CORE50 (n,3,128,128) and pretrained mobilenet
transform = transforms.Compose([
    #transforms.Lambda(lambda x: x.convert('RGB')),
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.repeat(3, 1, 1).to(device))
])

perm_mnist = PermutedMNIST(n_experiences=3, train_transform=transform, eval_transform=transform)
train_stream = perm_mnist.train_stream
test_stream = perm_mnist.test_stream

# Prepare for training & testing
criterion = CrossEntropyLoss()

# Continual learning strategy
cl_strategy = LatentReplay(criterion, train_mb_size=32, train_epochs=2,
                           eval_mb_size=32, device=device)

# train and test loop over the stream of experiences
results = []
for train_exp in train_stream:
    cl_strategy.train(train_exp)
    results.append(cl_strategy.eval(test_stream))


-- >> Start of training phase << --
-- Starting training on experience 0 (Task 0) from train stream --
100%|██████████| 1875/1875 [39:18<00:00,  1.26s/it]
Epoch 0 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.6677
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.7870
100%|██████████| 1875/1875 [20:52<00:00,  1.50it/s]
Epoch 1 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.2479
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9236
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 313/313 [01:16<00:00,  4.07it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task000/Exp000 = 0.4194
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.8597
-- Starting eval on experience 1 (Task 1) from test stream --
100%|██████████| 313/313 [01:18<00:00,  4.01it/s]
> Eval on experience 1 (Task 1) from test stream ended.
	Loss_Exp/eval_phas

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [4]:
# Config
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# model
model = SimpleMLP(num_classes=10)

# CL Benchmark Creation
perm_mnist = PermutedMNIST(n_experiences=3)
train_stream = perm_mnist.train_stream
test_stream = perm_mnist.test_stream

# Prepare for training & testing
optimizer = SGD(model.parameters(), lr=0.001, momentum=0.9)
criterion = CrossEntropyLoss()

# Continual learning strategy
cl_strategy = Naive(
    model, optimizer, criterion, train_mb_size=32, train_epochs=2,
    eval_mb_size=32, device=device)

# train and test loop over the stream of experiences
results = []
for train_exp in train_stream:
    cl_strategy.train(train_exp)
    results.append(cl_strategy.eval(test_stream))


-- >> Start of training phase << --
-- Starting training on experience 0 (Task 0) from train stream --
100%|██████████| 1875/1875 [08:38<00:00,  3.62it/s]  
Epoch 0 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.4914
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.8581
100%|██████████| 1875/1875 [00:29<00:00, 63.24it/s]
Epoch 1 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.2607
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9246
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 313/313 [00:03<00:00, 78.57it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task000/Exp000 = 0.1857
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.9467
-- Starting eval on experience 1 (Task 1) from test stream --
100%|██████████| 313/313 [00:04<00:00, 77.54it/s]
> Eval on experience 1 (Task 1) from test stream ended.
	Loss_Exp/eval_ph